## READ ME
- ランダム行列の生成は$\theta \sim U(0,2\pi),\; 
A = \begin{pmatrix} \sin(\theta) & - \cos(\theta) \\ \cos(\theta) & \sin(\theta) \end{pmatrix},\;
\Lambda = \begin{pmatrix} U(0,h)^2 & 0 \\ 0 & U(0,h)^2 \end{pmatrix},\;
$で$A,\Lambda$を生成し，$X = A \Lambda {}^t\!A$として正定値行列を生成

## ライブラリのimport

In [240]:
# !pip install mip 
from mip import Model, maximize, minimize, xsum
import numpy as np
from numpy import pi
from numpy.random import rand, randn
from numpy.linalg import solve, det
from scipy.linalg import sqrtm
import math


## 定数の設定

In [241]:
p = 2 # 行列の次元
N = 400 # データ数
lam = 0.1 # 正則化パラメータ
delta = 10.0
epsilon = 1.0
h = 3.5 #U(-h,h)

## Wasserstein距離を計算する関数

In [242]:
def wasserstein(x, y):
    z = sqrtm(sqrtm(x) @ y @ sqrtm(x))
    res = np.trace(x) + np.trace(y) - 2 * np.trace(z)
    return res

## Entropy正則化されたWasserstein距離を計算する関数

In [243]:
def reg_wasserstein(x, y):
    M_lam = sqrtm(lam * lam * np.eye(p) + sqrtm(x) @ y @ sqrtm(x))
    res = np.trace(x) + np.trace(y) - 2.0 * np.trace(M_lam) - 2.0 * lam * math.log(det(M_lam - lam * np.eye(p)),math.e) - 2.0 * lam * p * math.log(2.0 * math.pi * lam, math.e) - 4.0 * lam * p * math.log(2.0 * math.pi, math.e) - 2.0 * lam * p + 2.0 * lam * math.log(det(x @ y),math.e) + 4.0 * lam * p * (math.log(2.0 * math.pi, math.e) + 1.0)
    return res

In [244]:
## test
# p = 2
print(wasserstein(np.eye(p),np.eye(p)))

0.0


In [245]:
# lam = 10.0
print(reg_wasserstein(np.eye(p),np.eye(p)))

0.6058665937452075


## 乱数の生成と距離グラフの生成

### wasserstein

In [246]:
P = []
nbh_w = [[] for _ in range(N)]

i = 0
np.random.seed(123)
while i < N:
    theta = rand(1) * 2.0 * math.pi
    a = np.array([[math.sin(theta), - math.cos(theta)],[math.cos(theta), math.sin(theta)]])
    d_1 = rand(1) * h
    d1 = d_1 * d_1
    d_2 = rand(1) * h
    d2 = d_2 * d_2
    diag = np.array([[float(d1), 0.0],[0.0, float(d2)]])
    a = a.reshape((2,2)) 
    diag = diag.reshape((2,2))
    v = a @ diag @ a.T
    # B(I,delta)内になければskip
    if det(v) < 0.0001:
        continue 
    if wasserstein(np.eye(p),v) > delta :
        continue
    
    for j in range(0,i):
        w_dist = wasserstein(P[j],v)
        if w_dist < epsilon:
            nbh_w[i].append(j)
            nbh_w[j].append(i)
            
    # print(i)    
    nbh_w[i].append(i)
            
    i += 1
    P.append(v)

print(nbh_w)
print(len(P),len(nbh_w))
    

C:\Users\森雄大\AppData\Local\Temp\ipykernel_25716\1009827076.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a = np.array([[math.sin(theta), - math.cos(theta)],[math.cos(theta), math.sin(theta)]])
C:\Users\森雄大\AppData\Local\Temp\ipykernel_25716\1009827076.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  diag = np.array([[float(d1), 0.0],[0.0, float(d2)]])


[[0, 4, 5, 8, 10, 11, 18, 22, 29, 32, 34, 36, 40, 43, 56, 57, 58, 64, 69, 74, 76, 79, 81, 82, 84, 86, 88, 102, 106, 109, 113, 115, 116, 124, 126, 129, 130, 131, 141, 142, 144, 147, 151, 155, 159, 171, 172, 175, 187, 193, 194, 199, 206, 207, 213, 215, 220, 224, 225, 234, 245, 253, 258, 259, 261, 265, 266, 268, 272, 273, 277, 287, 292, 293, 294, 295, 303, 305, 306, 309, 311, 313, 317, 322, 323, 327, 330, 334, 337, 338, 339, 341, 344, 346, 356, 359, 360, 361, 368, 369, 380, 381, 385, 386, 397], [1, 2, 3, 6, 9, 14, 17, 20, 31, 33, 38, 41, 53, 54, 59, 69, 73, 77, 87, 100, 101, 102, 117, 133, 139, 144, 148, 152, 157, 165, 168, 170, 176, 177, 178, 184, 186, 191, 198, 204, 205, 219, 222, 223, 226, 239, 241, 242, 243, 247, 248, 260, 275, 280, 283, 284, 285, 289, 291, 298, 301, 302, 304, 310, 312, 314, 315, 316, 318, 319, 320, 342, 370, 379, 388, 390, 392, 396, 399], [1, 2, 6, 7, 9, 10, 17, 20, 28, 31, 33, 38, 41, 46, 54, 57, 59, 68, 69, 70, 73, 77, 87, 100, 101, 102, 110, 117, 119, 122, 133, 13

### regularized wasserstein

In [247]:
P = []
nbh_rw = [[] for _ in range(N)]


i = 0
np.random.seed(123)
while i < N:
    theta = rand(1) * 2.0 * math.pi
    a = np.array([[math.sin(theta), - math.cos(theta)],[math.cos(theta), math.sin(theta)]])
    d_1 = rand(1) * 2.0
    d1 = d_1 * d_1
    d_2 = rand(1) * 2.0
    d2 = d_2 * d_2
    diag = np.array([[float(d1), 0.0],[0.0, float(d2)]])
    a = a.reshape((2,2)) 
    diag = diag.reshape((2,2))
    v = a @ diag @ a.T
    # B(I,delta)内になければskip
    if reg_wasserstein(np.eye(p),v) > delta:
        continue
    
    for j in range(0,i):
        rw_dist = reg_wasserstein(P[j],v)
        if rw_dist < epsilon:
            nbh_rw[i].append(j)
            nbh_rw[j].append(i)
            
    # print(i)    
    nbh_rw[i].append(i)
            
    i += 1
    P.append(v)

print(nbh_rw)
print(len(P),len(nbh_rw))
    

C:\Users\森雄大\AppData\Local\Temp\ipykernel_25716\1771444837.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a = np.array([[math.sin(theta), - math.cos(theta)],[math.cos(theta), math.sin(theta)]])
C:\Users\森雄大\AppData\Local\Temp\ipykernel_25716\1771444837.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  diag = np.array([[float(d1), 0.0],[0.0, float(d2)]])


[[0, 4, 5, 8, 9, 10, 11, 13, 16, 18, 22, 25, 27, 29, 32, 34, 36, 40, 45, 46, 51, 54, 58, 59, 60, 66, 68, 69, 70, 71, 75, 76, 78, 81, 83, 84, 86, 88, 89, 90, 91, 92, 99, 104, 106, 108, 111, 114, 115, 117, 118, 119, 120, 122, 126, 128, 129, 131, 132, 133, 136, 138, 143, 144, 146, 149, 153, 156, 157, 158, 161, 164, 168, 171, 174, 175, 176, 177, 178, 180, 182, 183, 185, 186, 188, 190, 195, 196, 197, 199, 202, 203, 209, 210, 214, 216, 218, 223, 224, 227, 229, 230, 232, 236, 237, 238, 239, 244, 250, 258, 260, 263, 264, 265, 266, 268, 270, 271, 273, 277, 278, 282, 285, 291, 292, 297, 298, 299, 300, 303, 306, 307, 308, 310, 311, 312, 314, 316, 318, 319, 321, 322, 325, 327, 328, 330, 331, 332, 333, 335, 336, 339, 342, 343, 344, 345, 346, 349, 352, 354, 356, 361, 363, 364, 366, 367, 368, 372, 375, 376, 383, 388, 389, 390, 391, 393, 394, 395], [1, 2, 3, 9, 10, 11, 13, 14, 16, 17, 18, 20, 25, 26, 27, 31, 33, 37, 38, 42, 45, 55, 56, 59, 61, 71, 75, 79, 83, 89, 99, 100, 102, 103, 104, 108, 119, 121,

## Covering number

### Wasserstein

In [248]:
from mip import Model, maximize, minimize, xsum

In [249]:
m = Model()  # 数理モデル
x = []
# 変数
for i in range(N):
    s = "x"+str(i)
    x.append(m.add_var(s, lb=0, var_type="I"))

# 目的関数
z = x[0]
for i in range(1,N):
    z += x[i]

m.objective = minimize(z)
# m.objective = maximize(100 * x + 100 * y)
# 制約条件
for i in range(N):
    b = 0
    for j in nbh_w[i]:
        b += x[j]
    m += b >= 1
    
m.optimize()  # ソルバーの実行
print(m.objective.x)

# print("obj", m.objective.x, "x", x.x, "y", y.x)

13.0


### Regularized Wasserstein

In [250]:
m = Model()  # 数理モデル
x = []
# 変数
for i in range(N):
    s = "x"+str(i)
    x.append(m.add_var(s, lb=0, var_type="I"))

# 目的関数
z = x[0]
for i in range(1,N):
    z += x[i]

m.objective = minimize(z)
# m.objective = maximize(100 * x + 100 * y)
# 制約条件
for i in range(N):
    b = 1
    for j in nbh_rw[i]:
        b += x[j]
    m += b >= 2
    
m.optimize()  # ソルバーの実行
print(m.objective.x)

39.0
